<a href="https://colab.research.google.com/github/nazeehgh/CostSensetive_Telemarketing_MOGA/blob/main/CostSensetive_Telemarketing_bank_additional_Gen_FS_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### If you are using Colab you may connect Google Drive to manage files and outputs
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
### (Step 1)
#%reset
import numpy as np
import pandas as pd

FolderPath = '/gdrive/MyDrive/Colab_WS/Telemarketing/'
FileID = "May2023_5-Cost-" # ID for the saved file, it can be updated manually here, #FOR DOCUMENTATION PURPOSES
ExpID = ""
classifierName = "Telemarketing_bank-additional-full_XGBOOST_GenTunFS"  #FOR DOCUMENTATION PURPOSES
fileName = classifierName + "_" + FileID +".csv"
scale_pos_weight = 1
RSeed = 723 # preset random seed value to ensure reproducibility

# Initial all runs summaries
AllBestSolSummary = pd.DataFrame({
    "ExpID":[],
    "BestFitness":[],
    "FitnessFunction":[],
    "numberOfParentstostart":[],
    "numberOfParentsMating":[],
    "numberOfGenerations":[],
    "numberOfXGBParameters":[],
    "Perc2Sel":[],
    "NumFeat2Select":[],
    "NumOfSelectedFeatures":[],
    "pos_label":[],
    "objective":[],
    "learning_rate":[],
    "n_estimators":[],
    "max_depth":[],
    "min_child_weight":[],
    "gamma":[],
    "subsample":[],
    "colsample_bytree":[],
    "scale_pos_weight": [],
    "seed":[],
    "GADuration":[],
    "gen_BestSelectedFeaturesIndices":[],
    "gen_BestFitnessHistory":[],

    })

In [ ]:
### (Step 2)
### Read dataset file
# pd.set_option("display.max_colwidth", 200)

df = pd.read_csv(FolderPath+'bank-additional-full.csv', delimiter=';', header=0)

print("Dataset has {} entries and {} features".format(*df.shape))

X_Original = df.drop(columns=['y'])
X = X_Original.copy()
y = df['y']  # Label, target class

# Replace categorical data with numeric data (Convert Numeric to Categorical)
y = y.replace({'yes': 1, 'no': 0})

pos_label=1  ### Set Positive Class Label
neg_label=0   ### Set Negative Class Label

df["y"].value_counts()

Dataset has 41188 entries and 21 features


no     36548
yes     4640
Name: y, dtype: int64

In [ ]:
yes2noRatio= df["y"].value_counts()["no"] / df["y"].value_counts()["yes"]
yes2noRatio

7.876724137931035

In [ ]:
### (Step 3)
### Hot encode categorical attributes, by Nazeeh Ghatasheh

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

#  X.drop(['duration'] ,axis=1, inplace=True)

def HotEncode(feat):
  encoder=OneHotEncoder(sparse=False)
  X_encoded = pd.DataFrame (encoder.fit_transform(X_Original[[feat]]))
  X_encoded.columns = encoder.get_feature_names_out([feat])
  return X_encoded

for feat in X.columns:
  if X[feat].dtype == object:   # Hot encode all object data types
    X_encoded = HotEncode(feat)
    X_encoded = X_encoded.astype(int)
    X = pd.concat([X, X_encoded ], axis=1)
    X.drop([feat] ,axis=1, inplace=True)
    print(feat, ' Hot encoded')

print('X shape after hot encoding: ',X.shape)

In [ ]:
X.shape, X_Original.shape, y.shape

((41188, 63), (41188, 20), (41188,))

In [ ]:
FeaturesNames = X.columns
FeaturesNames

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oc

In [ ]:
## after removing duration
FeaturesNames = X.columns
FeaturesNames

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oc

In [ ]:
X.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,...,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911,...,0.017432,0.013839,0.190031,0.206711,0.209357,0.196416,0.197485,0.103234,0.863431,0.033335
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528,...,0.130877,0.116824,0.392330,0.404951,0.406855,0.397292,0.398106,0.304268,0.343396,0.179512
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
### (Step 5)

### randomize features over parents. used to generate reduced features (random feature selecton)
import random
random.seed(RSeed)

# total number of features in the dataset
numberOfDatasetFeatures = X.shape[1]


# Define the number of 1'(i.e. K )s in a randomly distributed array of size N
def rand_bin_array(K, N):
    arr = np.zeros(N, dtype=int)
    arr[:K]  = 1
    np.random.shuffle(arr)
    return arr

def get_RandomOnesArray(TotalnumberOfDatasetFeatures):
  NumberofSelFeatures = random.randint(1,TotalnumberOfDatasetFeatures)
  #NumberofSelFeatures = int(np.ceil(0.5 * TotalnumberOfDatasetFeatures)) #Percentage of features number to be selected
  BinaryFeaturesList = rand_bin_array(NumberofSelFeatures,TotalnumberOfDatasetFeatures)
  return NumberofSelFeatures, BinaryFeaturesList

def select_FeaturesIndices(TotalnumberOfDatasetFeatures, NumberofFeatures2Select):
  BinaryFeaturesList = rand_bin_array(NumberofFeatures2Select,TotalnumberOfDatasetFeatures)
  SelectedFeaturesIndices = np.where(BinaryFeaturesList == 1)[0]
  return SelectedFeaturesIndices


def get_ValuesDropDuplicates(CompleteList):
  return list(dict.fromkeys(CompleteList))

In [ ]:
### (Step 6)  The Modified GA
### [V2] XGBoost Parameter Tuning and Feature Selection using (Genetic Alg...)
### Based on https://github.com/mjain72/Hyperparameter-tuning-in-XGBoost-using-genetic-algorithm/blob/master/geneticXGboost.py

import numpy as np
import pandas as pd
import random
import xgboost as xgb
import matplotlib.pyplot as plt

random.seed(RSeed)
np.random.seed(RSeed)


def initilialize_poplulation(numberOfParents):
    numberOfDatasetFeatures = X.shape[1] # total number of features in the dataset

    learningRate = np.empty([numberOfParents, 1])
    nEstimators = np.empty([numberOfParents, 1], dtype = np.uint8)
    maxDepth = np.empty([numberOfParents, 1], dtype = np.uint8)
    minChildWeight = np.empty([numberOfParents, 1])
    gammaValue = np.empty([numberOfParents, 1])
    subSample = np.empty([numberOfParents, 1])
    colSampleByTree =  np.empty([numberOfParents, 1])

    features = np.empty([numberOfParents,NumFeat2Select], dtype=int)

    for i in range(numberOfParents):
        #print("Parent ", i, end =" ")
        learningRate[i] = round(random.uniform(0.01, 1), 2)
        nEstimators[i] = random.randrange(10, 1500, step = 25)
        maxDepth[i] = int(random.randrange(1, 10, step= 1))
        minChildWeight[i] = round(random.uniform(0.01, 10.0), 2)
        gammaValue[i] = round(random.uniform(0.01, 10.0), 2)
        subSample[i] = round(random.uniform(0.01, 1.0), 2)
        colSampleByTree[i] = round(random.uniform(0.01, 1.0), 2)
        SelFeaturesIndList = select_FeaturesIndices(numberOfDatasetFeatures, NumFeat2Select)
        features[i] = SelFeaturesIndList


        # Testing and Printing Outputs
        # print('Selected features names:', X.columns[SelFeaturesIndList])
        #print(" # Features ", NumberofSelectedFeatures, " | ", BinaryFeaturesList, end =" ")
        #print(" # Features", SelFeaturesIndList)
        #features[i] = random.sample(range(numberOfDatasetFeatures), feat_size) #random samples covering all the feature list
        # numOfFeat, FeatIndices = get_SelectedFeaturesIndices(numberOfDatasetFeatures)
        # features[i] = FeatIndices

    print()
    population = np.concatenate((learningRate, nEstimators, maxDepth, minChildWeight, gammaValue, subSample, colSampleByTree, features), axis= 1)
    population
    return population


#create fitness function that will predict F1_score
def fitness_f1score(y_true, y_pred):
    fitness = round((f1_score(y_true, y_pred, average='weighted')), 4)
    return fitness

### Added by Nazeeh Ghatasheh
from imblearn.metrics import geometric_mean_score
def fitness_GMean_score(y_true, y_pred):
  #fitness = round( geometric_mean_score(y_true, y_pred, average='weighted', pos_label=pos_label), 4)
  fitness = round( geometric_mean_score(y_true, y_pred, average='weighted'), 4)
  return fitness

#train the data annd find fitness score
def train_population(population):
    fScore = []
    for i in range(population.shape[0]):
        param = { 'objective':'binary:logistic',
              'learning_rate': population[i][0],
              'n_estimators': population[i][1],
              'max_depth': int(population[i][2]),
              'min_child_weight': population[i][3],
              'gamma': population[i][4],
              'subsample': population[i][5],
              'colsample_bytree': population[i][6],
              'scale_pos_weight': scale_pos_weight,
              'seed': RSeed}
        #get the indices of the selected features
        #SelectedFeatures = get_SelectedFeaturesIndices(population[i][7:])
        SelectedFeatures = get_ValuesDropDuplicates(population[i][7:])
        # SelectedFeatures = population[i][7:]
        # print(SelectedFeatures)

        dMatrixTrain = xgb.DMatrix(X_train.iloc[:,SelectedFeatures], label=y_train) #create Dmatrix
        dMatrixtest = xgb.DMatrix(X_test.iloc[:,SelectedFeatures], label=y_test)

        #print("ind", len(SelectedFeatures), SelectedFeatures)

        xgbT = xgb.train(param, dMatrixTrain, num_XGB_round)
        preds = xgbT.predict(dMatrixtest)
        preds = preds>0.5
        fScore.append(fitness_GMean_score(y_test, preds))
    return fScore


#select parents for mating
def new_parents_selection(population, fitness, numParents):
    selectedParents = np.empty((numParents, population.shape[1])) #create an array to store fittest parents

    #find the top best performing parents
    for parentId in range(numParents):
        bestFitnessId = np.where(fitness == np.max(fitness))
        bestFitnessId  = bestFitnessId[0][0]
        selectedParents[parentId, :] = population[bestFitnessId, :]
        fitness[bestFitnessId] = -1 #set this value to negative, in case of F1-score, so this parent is not selected again
    return selectedParents


#### Modified Crossover to include features and parameters
def crossover_ParamFeat_uniform(parents, childrenSize):

    crossoverPointIndex_par = np.arange(0, np.uint8(numberOfXGBParameters), 1, dtype= np.uint8) #get all the indices of the parameters
    crossoverPointIndex_feat = np.arange(numberOfXGBParameters, np.uint16(childrenSize[1]), 1, dtype= np.uint16) #get all the indices of the features

    #xgb Parameters Crossover Indices
    crossoverPointIndex1 = get_ValuesDropDuplicates(np.random.randint(0, np.uint8(numberOfXGBParameters), np.uint8(numberOfXGBParameters/2)) ) # select about half  of the indices randomly
    crossoverPointIndex2 = np.array(list(set(crossoverPointIndex_par) - set(crossoverPointIndex1))) #select leftover indices

    #Features Crossover Indices
    crossoverPointIndex1_feat = get_ValuesDropDuplicates(np.random.randint(numberOfXGBParameters, np.uint16(childrenSize[1]), np.uint16(NumFeat2Select/2)) ) # select half  of the indices randomly
    crossoverPointIndex2_feat = np.array(list(set(crossoverPointIndex_feat) - set(crossoverPointIndex1_feat))) #select leftover indices


    #print("Crossover Par:", crossoverPointIndex_par, crossoverPointIndex1, crossoverPointIndex2, childrenSize)
    #print("Crossover Feat:", crossoverPointIndex_feat, crossoverPointIndex1_feat, crossoverPointIndex2_feat, childrenSize)
    #print("Children indices;", crossoverPointIndex1, crossoverPointIndex2, crossoverPointIndex1_feat, crossoverPointIndex2_feat)


    children = np.empty(childrenSize, dtype=float)


    for i in range(childrenSize[0]):

        #find parent 1 index
        parent1_index = i%parents.shape[0]
        #find parent 2 index
        parent2_index = (i+1)%parents.shape[0]
        #insert parameters based on random selected indexes in parent 1
        children[i, crossoverPointIndex1] = parents[parent1_index, crossoverPointIndex1]
        #insert parameters based on random selected indexes in parent 1
        children[i, crossoverPointIndex2] = parents[parent2_index, crossoverPointIndex2]

        #insert features based on random selected indices in parent 1
        children[i, crossoverPointIndex1_feat] = parents[parent1_index, crossoverPointIndex1_feat]
        #insert features based on random selected indices in parent 1
        children[i, crossoverPointIndex2_feat] = parents[parent2_index, crossoverPointIndex2_feat]
    return children




def mutation(crossover, numberOfXGBParameters, NumFeat2Select):
    #Define minimum and maximum values allowed for each parameter

    minMaxValue = np.zeros((numberOfXGBParameters, 2))

    ### Booster Parameters
    minMaxValue[0:] = [0.01, 2.0] #min/max learning rate
    minMaxValue[1, :] = [10, 500] #min/max n_estimator
    minMaxValue[2, :] = [3, 10] #min/max depth
    minMaxValue[3, :] = [0, 10.0] #min/max child_weight
    minMaxValue[4, :] = [0.01, 10.0] #min/max gamma
    minMaxValue[5, :] = [0.5, 1.0] #min/maxsubsample
    minMaxValue[6, :] = [0.5, 1.0] #min/maxcolsample_bytree

    # (Parameters) Mutation changes a single gene in each offspring randomly.
    mutationValue = 0
    parameterSelect = np.random.randint(0, numberOfXGBParameters, 1)
    #print(parameterSelect)
    #print("mutated parameter gene @", parameterSelect)
    if parameterSelect == 0: #learning_rate
        mutationValue = round(np.random.uniform(-0.5, 0.5), 2)
    if parameterSelect == 1: #n_estimators
        mutationValue = np.random.randint(-200, 200, 1)
    if parameterSelect == 2: #max_depth
        mutationValue = np.random.randint(-5, 5, 1)
    if parameterSelect == 3: #min_child_weight
        mutationValue = round(np.random.uniform(5, 5), 2)
    if parameterSelect == 4: #gamma
        mutationValue = round(np.random.uniform(-2, 2), 2)
    if parameterSelect == 5: #subsample
        mutationValue = round(np.random.uniform(-0.5, 0.5), 2)
    if parameterSelect == 6: #colsample
        mutationValue = round(np.random.uniform(-0.5, 0.5), 2)

    # (Features) Mutation changes a single gene in each offspring randomly.
    mutationValue_feat = 0
    parameterSelect_feat = np.random.randint(numberOfXGBParameters, numberOfXGBParameters+NumFeat2Select, 1) # select feature index randomly to be mutated
    # print("mutated feature gene @", parameterSelect_feat)


    #indtroduce mutation by changing one parameter, and set to max or min if it goes out of range
    for idx in range(crossover.shape[0]):
        crossover[idx, parameterSelect] = crossover[idx, parameterSelect] + mutationValue
        if(crossover[idx, parameterSelect] > minMaxValue[parameterSelect, 1]):
            crossover[idx, parameterSelect] = minMaxValue[parameterSelect, 1]
        if(crossover[idx, parameterSelect] < minMaxValue[parameterSelect, 0]):
            crossover[idx, parameterSelect] = minMaxValue[parameterSelect, 0]

        #Mutate features by replacing a gene with a random dataset feature. Such that the new feature is not already found in the current chromosome
        crossover[idx, parameterSelect_feat]= random.choice([x for x in range(numberOfDatasetFeatures) if x not in crossover[idx, numberOfXGBParameters:]])
        #print(crossover[idx])
        #print("mutated features gene @", parameterSelect_feat, " to be: ", crossover[idx, parameterSelect_feat])
    return crossover


'''
This function will allow us to genrate the heatmap for various parameters and fitness to visualize
how each parameter and fitness changes with each generation
'''
def plot_parameters(numberOfGenerations, numberOfParents, parameter, parameterName):
    #inspired from https://matplotlib.org/gallery/images_contours_and_fields/image_annotated_heatmap.html
    generationList = ["Gen {}".format(i) for i in range(numberOfGenerations+1)]
    populationList = ["Parent {}".format(i) for i in range(numberOfParents)]

    fig, ax = plt.subplots()
    im = ax.imshow(parameter, cmap=plt.get_cmap('YlOrBr'))
    # show ticks
    ax.set_xticks(np.arange(len(populationList)))
    ax.set_yticks(np.arange(len(generationList)))

    # show labels
    ax.set_xticklabels(populationList)
    ax.set_yticklabels(generationList)

    # set ticks at 45 degrees and rotate around anchor
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")


    # insert the value of the parameter in each cell
    for i in range(len(generationList)):
        for j in range(len(populationList)):
            text = ax.text(j, i, parameter[i, j],
                           ha="center", va="center", color="k")

    ax.set_title("Change in the value of " + parameterName)
    fig.tight_layout()
    plt.show()


##############################################################################################
'''
---------------------------------------------------------------------------------------------
            MAIN CODE Below
---------------------------------------------------------------------------------------------
'''
############### Main Code for Parameters Optimization and Feature Selection using the modified GA #########

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = RSeed)


numberOfDatasetFeatures = X.shape[1] # total number of features in the dataset

scale_pos_weight = 8 #@param {type:"integer"}
num_XGB_round = 100 #@param {type:"integer"}

### GA Configuration.....
Perc2Sel = 0.30 #@param {type:"number"}
CrossoverRatio = 0.2 #@param {type:"number"}
NumFeat2Select = int(np.ceil(Perc2Sel * numberOfDatasetFeatures)) # Maximum number of features to select

numberOfParents = 100 #@param {type:"integer"}  #number of parents to start
# numberOfParents = int(np.ceil(2 * NumFeat2Select))
numberOfParentsMating = int(np.ceil(CrossoverRatio * numberOfParents))
numberOfXGBParameters = 7  #Number of algorithm parameters to be optimized
numberOfParameters = numberOfXGBParameters + NumFeat2Select # total number of parameters to be optimized and max number of  features to be selected
numberOfGenerations = 30 #@param {type:"integer"} #number of genrations that will be created



ExpID = "F" + str(int(Perc2Sel*100)) + "-P" + str(numberOfParents) + "-C" + str(numberOfParentsMating) + "-G" + str(numberOfGenerations) + "-PCost" + str(scale_pos_weight) + "-BR" + str(num_XGB_round)
print("ExpID:", ExpID, " NumFeat2Select:", NumFeat2Select)
print('numberOfGenerations: ',numberOfGenerations, ' numberOfXGBParameters:',numberOfXGBParameters, ' scale_pos_weight: ', scale_pos_weight)

GAHyperParameterTuningConf= pd.DataFrame({"pos_label":[pos_label],
                                          "FitnessFunction":["GMean"],
                   "numberOfParentstostart":[numberOfParents],
                   "numberOfParentsMating":[numberOfParentsMating],
                   "numberOfGenerations":[numberOfGenerations],
                   "numberOfXGBParameters":[numberOfXGBParameters],
                   "NumFeat2Select":[NumFeat2Select],
                   })

import time
t1 = time.perf_counter()

#define the population size
populationSize = (numberOfParents, numberOfParameters)

#initialize the population with randomly generated parameters
population = initilialize_poplulation(numberOfParents)

#define an array to store the fitness  hitory
fitnessHistory = np.empty([numberOfGenerations+1, numberOfParents])

#define an array to store the value of each parameter for each parent and generation
populationHistory = np.empty([(numberOfGenerations+1)*numberOfParents, numberOfParameters])

#insert the value of initial parameters to history
populationHistory[0:numberOfParents, :] = population

for generation in range(numberOfGenerations):
    print("<<<<< This is generation number %s " % (generation) ," >>>>>")
    #train the dataset and obtain fitness
    fitnessValue = train_population(population=population)

    fitnessHistory[generation, :] = fitnessValue

    #best score in the current iteration
    print('Best Fitness score in the this iteration = {}'.format(np.max(fitnessHistory[generation, :])))

    #survival of the fittest - take the top parents, based on the fitness value and number of parents needed to be selected
    parents = new_parents_selection(population=population, fitness=fitnessValue, numParents=numberOfParentsMating)

    #mate these parents to create children having parameters from these parents
    children = crossover_ParamFeat_uniform(parents=parents, childrenSize=(populationSize[0] - parents.shape[0], numberOfParameters))

    #add mutation to create genetic diversity
    ##children_mutated = mutation(children, numberOfParameters)
    children_mutated = mutation(children, numberOfXGBParameters, NumFeat2Select)


    '''
    We will create new population, which will contain parents that where selected previously based on the
    fitness score and rest of them  will be children
    '''
    population[0:parents.shape[0], :] = parents #fittest parents
    population[parents.shape[0]:, :] = children_mutated #children

    populationHistory[(generation+1)*numberOfParents : (generation+1)*numberOfParents+ numberOfParents , :] = population #store parent information


#Best solution from the final iteration
print("<<<<< Calculating fitnees of the last generation >>>>>")
#fitness = train_population(population=population, dMatrixTrain=xgDMatrix, dMatrixtest=xgbDMatrixTest, y_test=y_test)
fitness = train_population(population=population)
fitnessHistory[generation+1, :] = fitness
print('Best Fitness score of the last generation = {}'.format(np.max(fitnessHistory[generation+1, :])))


t2 = time.perf_counter()
GADuration = t2-t1
print('GA Running Time: ',GADuration, ' seconds')

#index of the best solution
bestFitnessIndex = np.where(fitness == np.max(fitness))[0][0]

# to store best fitness in each generation.
gen_BestFitnessHistory = []
for generation in range(numberOfGenerations+1):
  max = np.max(fitnessHistory[generation, :])
  gen_BestFitnessHistory.append(max)
print(gen_BestFitnessHistory)

gen_params = {}
gen_params['objective'] = 'binary:logistic'
gen_params['learning_rate'] = population[bestFitnessIndex][0]
gen_params['n_estimators'] = int(population[bestFitnessIndex][1])
gen_params['max_depth'] = int(population[bestFitnessIndex][2])
gen_params['min_child_weight'] = population[bestFitnessIndex][3]
gen_params['gamma'] = population[bestFitnessIndex][4]
gen_params['subsample'] = population[bestFitnessIndex][5]
gen_params['colsample_bytree'] = population[bestFitnessIndex][6]
gen_params['scale_pos_weight'] = scale_pos_weight
gen_params['seed'] = RSeed


#gen_BestSelectedFeaturesIndices = get_SelectedFeaturesIndices(population[bestFitnessIndex][7:])
gen_BestSelectedFeaturesIndices = get_ValuesDropDuplicates(population[bestFitnessIndex][7:])
gen_BestSelectedFeaturesIndices = [int(item) for item in gen_BestSelectedFeaturesIndices]
print('Best selected features indices:', gen_BestSelectedFeaturesIndices)
print('Best selected features names:', X.columns[gen_BestSelectedFeaturesIndices])
#(gen_BestSelectedFeaturesIndices == 1).sum()


####  Reporting and Logs

#Best fitness
print("Best fitness =", fitness[bestFitnessIndex])
#print("Number of Selected Fetures =", gen_BestSelectedFeaturesIndices.size)
print("Number of Selected Fetures =", len(gen_BestSelectedFeaturesIndices))

#Best parameters
print("Best parameters are:")
#print('learning_rate', population[bestFitnessIndex][0])
#print('n_estimators', population[bestFitnessIndex][1])
#print('max_depth', int(population[bestFitnessIndex][2]))
#print('min_child_weight', population[bestFitnessIndex][3])
#print('gamma', population[bestFitnessIndex][4])
#print('subsample', population[bestFitnessIndex][5])
#print('colsample_bytree', population[bestFitnessIndex][6])
print(gen_params)

#visualize the change in fitness of the various generations and parents
#plot_parameters(numberOfGenerations, numberOfParents, fitnessHistory, "fitness (F1-score)")

#Look at individual parameters change with generation
#Create array for each parameter history (Genration x Parents)

learnigRateHistory = populationHistory[:, 0].reshape([numberOfGenerations+1, numberOfParents])
nEstimatorHistory = populationHistory[:, 1].reshape([numberOfGenerations+1, numberOfParents])
maxdepthHistory = populationHistory[:, 2].reshape([numberOfGenerations+1, numberOfParents])
minChildWeightHistory = populationHistory[:, 3].reshape([numberOfGenerations+1, numberOfParents])
gammaHistory = populationHistory[:, 4].reshape([numberOfGenerations+1, numberOfParents])
subsampleHistory = populationHistory[:, 5].reshape([numberOfGenerations+1, numberOfParents])
colsampleByTreeHistory = populationHistory[:, 6].reshape([numberOfGenerations+1, numberOfParents])

#featCountHistory = np.empty([populationHistory.shape[0], 1])
#for p in range(populationHistory.shape[0]):
#  featCountHistory[p] = (populationHistory[p, 7:] == 1).sum()
#SelectedFeaturesCountHistory = featCountHistory[:, :].reshape([numberOfGenerations+1, numberOfParents])


### Save to files
CurrentBestSolSummary = pd.DataFrame({
    "ExpID":[ExpID],
    "BestFitness":[fitness[bestFitnessIndex]],
    "FitnessFunction":["GMean"],
    "numberOfParentstostart":[numberOfParents],
    "numberOfParentsMating":[numberOfParentsMating],
    "numberOfGenerations":[numberOfGenerations],
    "numberOfXGBParameters":[numberOfXGBParameters],
    "Perc2Sel":[Perc2Sel],
    "NumFeat2Select":[NumFeat2Select],
    "NumOfSelectedFeatures":[len(gen_BestSelectedFeaturesIndices)],
    "pos_label":[pos_label],
    "objective":[gen_params['objective']],
    "learning_rate":[gen_params['learning_rate']],
    "n_estimators":[gen_params['n_estimators']],
    "max_depth":[gen_params['max_depth']],
    "min_child_weight":[gen_params['min_child_weight']],
    "gamma":[gen_params['gamma']],
    "subsample":[gen_params['subsample']],
    "colsample_bytree":[gen_params['colsample_bytree']],
    "scale_pos_weight": [gen_params['scale_pos_weight']],
    "seed":[gen_params['seed']],
    "GADuration":[GADuration],
    "gen_BestSelectedFeaturesIndices":[gen_BestSelectedFeaturesIndices],
    "gen_BestFitnessHistory":[gen_BestFitnessHistory],
    "gen_BestSelectedFeaturesNames":[X.columns[gen_BestSelectedFeaturesIndices]]

    })
AllBestSolSummary = AllBestSolSummary.append(CurrentBestSolSummary, ignore_index=True)
AllBestSolSummaryfileName = classifierName + "_" + FileID + "_AllBestSolSummary" + ".csv"
FileLocation = FolderPath + AllBestSolSummaryfileName
AllBestSolSummary.to_csv(FileLocation)
#print(CurrentBestSolSummary)
#print(AllBestSolSummary)


#### Extra Reporting
#populationHistoryfileName = ExpID + "_" + classifierName + "_" + FileID + "_PopulationHistory" + ".csv"
#np.savetxt(populationHistoryfileName, populationHistory[:,0:7], delimiter=",") #Save Classifier Parameters
#fitnessHistoryfileName = ExpID + "_" + classifierName + "_" + FileID + "_FitnessHistory" + ".csv"
#np.savetxt(fitnessHistoryfileName, fitnessHistory, delimiter=",")
#SelectedFeaturesIndicesHistoryfileName = ExpID + "_" + classifierName + "_" + FileID + "_SelectedFeaturesIndices" + ".csv"
#np.savetxt(SelectedFeaturesIndicesHistoryfileName, populationHistory[:,7:], delimiter=",")
#SelectedBinaryFeaturesHistoryfileName = classifierName + "_" + FileID + "_SelectedBinaryFeatures" + ".csv"
#np.savetxt(SelectedBinaryFeaturesHistoryfileName, populationHistory[:,7:], delimiter=",")

#generate heatmap for each parameter

#plot_parameters(numberOfGenerations, numberOfParents, learnigRateHistory, "learning rate")
#plot_parameters(numberOfGenerations, numberOfParents, nEstimatorHistory, "n_estimator")
#plot_parameters(numberOfGenerations, numberOfParents, maxdepthHistory, "maximum depth")
#plot_parameters(numberOfGenerations, numberOfParents, minChildWeightHistory, "minimum child weight")
#plot_parameters(numberOfGenerations, numberOfParents, gammaHistory, "gamma")
#plot_parameters(numberOfGenerations, numberOfParents, subsampleHistory, "subsample")
#plot_parameters(numberOfGenerations, numberOfParents, colsampleByTreeHistory, "col sample by history")
#plot_parameters(numberOfGenerations, numberOfParents, SelectedFeaturesCountHistory, "Number of Selected Features")

ExpID: F30-P100-C20-G30-PCost8-BR100  NumFeat2Select: 19
numberOfGenerations:  30  numberOfXGBParameters: 7  scale_pos_weight:  8

<<<<< This is generation number 0   >>>>>
[10:31:28] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:30] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:32] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:33] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:34] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:36] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:37] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:37] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:40] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:31:42] WARNIN

In [ ]:
### Load Configurations and Selected Features From File
data = pd.read_csv(FolderPath+"Telemarketing_GAFS-Cost_Results - GA-FS-Sensitivity.csv", delimiter=",")


def get_gen_Params(ind):
  gen_params = {}
  gen_params['objective'] = data['objective'][ind]
  gen_params['learning_rate'] = data['learning_rate'][ind]
  gen_params['n_estimators'] = int(data['n_estimators'][ind])
  gen_params['max_depth'] = int(data['max_depth'][ind])
  gen_params['min_child_weight'] = data['min_child_weight'][ind]
  gen_params['gamma'] = data['gamma'][ind]
  gen_params['subsample'] = data['subsample'][ind]
  gen_params['colsample_bytree'] = data['colsample_bytree'][ind]
  gen_params['scale_pos_weight'] = data['scale_pos_weight'][ind]
  gen_params['seed'] = RSeed
  return gen_params

def get_GAHyperParameterTuningConf(ind):
  GAHyperParameterTuningConf = pd.DataFrame({"pos_label":[data['pos_label'][ind]],
                                             "FitnessFunction":[data['FitnessFunction'][ind]],
                                             "BestFitness":[data['BestFitness'][ind]],
                   "numberOfParentstostart":[int(data['numberOfParentstostart'][ind])],
                   "numberOfParentsMating":[int(data['numberOfParentsMating'][ind])],
                   "numberOfGenerations":[int(data['numberOfGenerations'][ind])],
                   "numberOfXGBParameters":[int(data['numberOfXGBParameters'][ind])],
                   "Perc2Sel":[data['Perc2Sel'][ind]],
                   "NumFeat2Select":[int(data['NumFeat2Select'][ind])],
                   "NumOfSelectedFeatures":[int(data['NumOfSelectedFeatures'][ind])],
                   })
  return GAHyperParameterTuningConf

def convert_txt2list(txt, toInt, toFloat):
  converted_list=[]
  for line in txt:
    line = str(line)
    #print(line)
    line = line.replace('[','')
    line = line.replace(']','')
    if toInt:
      converted_list.append([int(float(n)) for n in line.split(',')])
    elif toFloat:
       converted_list.append([float(n) for n in line.split(',')])
    else:
       converted_list.append([n for n in line.split(',')])

  return converted_list

data.shape


(38, 27)

In [ ]:
### (Step 7)

#Calssifier Test No 2. #### XGBoost Runs and CV Setup + Using GA Selected Features (Main Code) ########

### TiPS https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from xgboost import plot_tree
from matplotlib.pylab import rcParams
import time

###### Tuning Parameters After GA Optimizatoin Step >>>>> uncomment the following line if you load configurations manually
TuningParameters = GAHyperParameterTuningConf  # Get GA Hyperparameter tuning configuration

### To load saved configurations uncomment the following
# saved_conf_ind = 5 #@param {type:"integer"}
# TuningParameters = get_GAHyperParameterTuningConf(saved_conf_ind)
# gen_params = get_gen_Params(saved_conf_ind)
# features_list_txt = data["gen_BestSelectedFeaturesIndices"]
# features_list= convert_txt2list(features_list_txt, True, False)
# gen_BestSelectedFeaturesIndices = features_list[saved_conf_ind]
# ExpID = data['ExpID'][saved_conf_ind]
# print("Loaded Experiment [" , ExpID , "], Index [" , saved_conf_ind , "]" )



######### Fill in XGBOOST tuned parameters manually ##################################
# XGBParameters = pd.DataFrame({"Best_fitness":[0.0],
#                     "learning_rate":[0.41],
#                     "n_estimators":[100],
#                     "max_depth":[5],
#                     "min_child_weight":[0.15],
#                     "gamma":[1.06],
#                     "subsample":[0.99],
#                     "colsample_bytree":[0.55]
#                     })




pd.set_option('display.max_columns', None)
print("Gen. TuningParameters:\n",TuningParameters)
#print("XGBParameters:\n", XGBParameters)
print("\nClassification POS Label: ", pos_label, ", NEG Label: ", neg_label, "\n")
print(gen_params, "\n")
classifier = XGBClassifier(**gen_params)


###### Cross Validation Code by Nazeeh
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict, cross_val_score
from sklearn.metrics import make_scorer, classification_report, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, f1_score
from sklearn.metrics import roc_curve, auc
import math
#from sklearn.metrics import plot_roc_curve
#from sklearn import metrics

number_of_folds = 10 #@param {}
skf = StratifiedKFold(n_splits=number_of_folds, shuffle=True)

# Variables for average classification report
originalclass = []
predictedclass = []


def classification_report_nz(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
#    print (classification_report(y_true, y_pred)) # print classification report
    print ('.', end=" ")
    return accuracy_score(y_true, y_pred) # return accuracy score

# Confusion Matrix entries for the scorer (a wrapper to return a single metric)
def tn(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred):
  return confusion_matrix(y_true, y_pred)[1, 1]

# define AUC scoring function
def custom_auc(y_true, y_pred):
  fpr, tpr, threshold = roc_curve(y_true, y_pred, pos_label=pos_label)
  #print('fpr: ',fpr,'\ntpr: ',tpr, '\nthreshold: ',threshold,'\n')
  return auc(fpr, tpr)



### Performance measures setup
scoring = {
           'tp': make_scorer(tp),
           'tn': make_scorer(tn),
           'fp': make_scorer(fp),
           'fn': make_scorer(fn),
           'prec': make_scorer(precision_score, pos_label=pos_label),
           'rec': make_scorer(recall_score, pos_label=pos_label),
           'f1': make_scorer(f1_score, pos_label=pos_label),
           #'prec_macro': 'precision_macro',
           #'rec_macro': make_scorer(recall_score, average='macro'),
           'accuracy':'accuracy',
           'roc_auc':'roc_auc',
           'my_auc': make_scorer(custom_auc, greater_is_better=True, needs_proba=True),
           'classification_report': make_scorer(classification_report_nz)
           }

number_of_runs = 50 #@param {type:"integer"}
run_scores=[]   ### Store the scores of all runs

### Store the scores of all runs the to CSV
df_AllTestRUNResultsLOG = pd.DataFrame({"ExpID":[],
                                   "classifier":[],
                                   "tp":[],
                                   "tn":[],
                                   "fp":[],
                                   "fn":[],
                                   "Run_Total_Accuracy":[],
                                   "runGMeans":[],
                                   "myAUC":[],
                                   "TPR":[],
                                   "TNR":[],
                                   "PPV":[],
                                   "FPR":[],
                                   "F1s":[],
                                   "NPV":[],
                                   "fit_time":[],
                                   "score_time":[],
                                   "estimator":[]
                    })


print('Validating Experiment: ' + ExpID)
gen_BestSelectedFeaturesNames = X.columns[gen_BestSelectedFeaturesIndices]
print('Selected features names:', gen_BestSelectedFeaturesNames)

for r in range(1,number_of_runs+1):
  print( ' _________________________\n',
  '| Run Number ', r, '/', number_of_runs, ':',
  '\n ^^^^^^^^^^^^^^^^^^^^^^^^^')
  originalclass = [] #reset
  predictedclass = [] #reset

  # print('Selected features names:', X.columns[gen_BestSelectedFeaturesIndices])
  scores = cross_validate(classifier, X.iloc[:,gen_BestSelectedFeaturesIndices], y, scoring=scoring,
                         cv=skf , return_train_score=False, return_estimator=True, n_jobs=-1)
  # rfc_fit = scores['estimator']
  # print(rfc_fit)
  # rcParams['figure.figsize'] = 80,50
  # plot_tree(rfc_fit[0]) # Plto the first model (rfc_fit[0])
  #### plot_tree(rfc_fit[0], num_trees=0, rankdir='LR')

  # plt.show()


#### https://github.com/dmlc/xgboost/blob/master/demo/guide-python/cross_validation.py
## https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html



  #### REPORTING ######
  # ####Save the model
  #### https://stackoverflow.com/questions/56904303/how-to-export-save-a-fitted-model-after-cross-validate-and-use-it-later-on-pan
  # ####scores['estimator']
  # # ###save it
  # from sklearn.externals import joblib
  # filename = os.path.join(savedir, 'final_model.joblib')
  # joblib.dump(rfc_fit,filename)
  # # #####load it
  # rfc_model2 = joblib.load(filename)
  # # ####Predict using the saved model:
  # predicted = rfc_model2.predict(X)

  ## Get Confusion Matrix entries
  print("\n------ Confusion Matrix, RUN: (",r,") --------\n" )
  tp= scores['test_tp']
  tn= scores['test_tn']
  fp= scores['test_fp']
  fn= scores['test_fn']
  myAUC = scores['test_my_auc']
  TPR = tp / (tp+fn) #Recall, Sensitivity
  TNR = tn / (tn+fp)  #Specificity (SPC), Selectivity
  Run_Total_Accuracy = (tp+tn)/(tp+tn+fp+fn)
  PPV = tp / (tp+fp) #Precision
  FPR = fp / (fp+tn) #Fall-out, probability of false alarm
  F1s= 2 * ( (TPR * PPV) / (TPR + PPV) )
  NPV = tn / (tn+fn)
  fit_time = scores['fit_time']
  score_time = scores['score_time']
  estimator = scores['estimator']


  runGMeans=[]
  for x in range(len(TPR)):
    runGMeans.append(math.sqrt(TPR[x]*TNR[x]))

  print("Run ",r, "TPs:",tp)
  print("Run ",r, "TNs:",tn)
  print("Run ",r, "FPs:",fp)
  print("Run ",r, "FNs:",fn)
  print("\n------ Selected Measures, RUN: (",r,") --------\n" )
  print("Run ",r, "Run Total Accuracy:", Run_Total_Accuracy )
  print("Run ",r, "TPRs:",TPR)
  print("Run ",r, "TNRs:",TNR)
  print("Run ",r, "GMEANs:", runGMeans )
  print("Run ",r, "AUC POS Class:",myAUC)
  print("Run ",r, "PPV:",PPV)
  print("Run ",r, "FPR:",FPR)
  print("Run ",r, "F1s:",F1s)
  print("Run ",r, "NPV:",NPV,'\n')
  print("Run ",r, "fit_time:",fit_time,)
  print("Run ",r, "score_time:",score_time,'\n')
  # print("Run ",r, "estimator:",estimator,'\n')



  df_TestRUNResults = pd.DataFrame({"ExpID":[r for _ in range(number_of_folds)],
                                   "classifier":classifierName,
                                   "tp":tp,
                                   "tn":tn,
                                   "fp":fp,
                                   "fn":fn,
                                   "Run_Total_Accuracy":Run_Total_Accuracy,
                                   "runGMeans":runGMeans,
                                   "myAUC":myAUC,
                                   "TPR":TPR,
                                   "TNR":TNR,
                                   "PPV":PPV,
                                   "FPR":FPR,
                                   "F1s":F1s,
                                   "NPV":NPV,
                                   "fit_time":fit_time,
                                   "score_time":score_time,
                                   "estimator":estimator
                                    })

  print(df_TestRUNResults.describe())
  df_AllTestRUNResultsLOG = df_AllTestRUNResultsLOG.append(df_TestRUNResults, ignore_index = True)

### Compute Total Statistics
print(df_AllTestRUNResultsLOG.describe())
print("Saving ", FolderPath + "Runs_Log/" + ExpID + '-' + classifierName, "_",FileID," FILES" )

# Save basic metrics and descriptions into files
df_AllTestRUNResultsLOG.to_csv(FolderPath + "Runs_Log/" + ExpID + '-' + classifierName + "_" + FileID +"_RunsLog"+".csv")
df_AllTestRUNResultsLOG.describe().to_csv(FolderPath + "Runs_Log/" + ExpID + '-' + classifierName + "_" + FileID +"_RunsSummary"+".csv")

file1 = open(FolderPath + "Runs_Log/" + ExpID + '-' + classifierName + "_" + FileID +"_Configurations"+".txt","w")
file1.write("FileID:"+ FileID + "\n")
file1.write("classifier: "+ classifierName + "\n")
file1.write("Tuning Parameters: "+ TuningParameters.to_string() + "\n")
file1.write("XGBoost Best Parameters: "+ str(gen_params) + "\n")
file1.write("gen_BestSelectedFeaturesIndices: "+ str(gen_BestSelectedFeaturesIndices) + "\n")
file1.write("gen_BestSelectedFeaturesNames: "+ str(gen_BestSelectedFeaturesNames) + "\n")
#file1.write("XGBoost Best Parameters: "+ XGBParameters.to_string() + "\n")
file1.write("Classification POS Label: "+ str(pos_label) + ", NEG Label:" + str(neg_label) +"\n")
file1.write("classifier Configurations: \n" + str(classifier.get_params) +"\n")
#file1.writelines(L)
file1.close()

print( '*****************Completed**********************')
df_AllTestRUNResultsLOG.describe()